## SecSSE : Body size 

In [1]:
## SecSSE : Body size

## I Data preparation

In [2]:
## I Data preparation

### Loading packages

In [3]:
### Loading packages

In [4]:
library("secsse")
library("stringr")
library("qpcR")

Le chargement a nécessité le package : ape

Le chargement a nécessité le package : MASS

Le chargement a nécessité le package : minpack.lm

Le chargement a nécessité le package : rgl

This build of rgl does not include OpenGL functions.  Use
 rglwidget() to display results, e.g. via options(rgl.printRglwidget = TRUE).

Le chargement a nécessité le package : robustbase

Le chargement a nécessité le package : Matrix



### Loading data

In [5]:
### Loading data

In [43]:
df<-read.csv("Path/to/your/file.tsv", sep="\t") # omit sep ="\t" for .csv files
phy<-read.nexus("Path/to/your/tree.tree")

In [44]:
df

,Consensus,Reproduction,Lower.Slope.Deep,Upper.Slope,Shelf,Reef,Coastal,Oceanic.coastal.pelagic,clust.num
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
Aculeola nigra,1,YV,No,Yes,Yes,No,No,No,1
Alopias pelagicus,3,Oo,No,No,Yes,Yes,No,Yes,2
Alopias superciliosus,3,Oo,No,No,No,No,Yes,Yes,2
Alopias vulpinus,3,Oo,No,No,Yes,No,Yes,Yes,2
Apristurus albisoma,1,O,Yes,No,No,No,No,No,3
Apristurus ampliceps,2,O,Yes,No,No,No,No,No,3
Apristurus aphyodes,1,O,Yes,No,No,No,No,No,3
Apristurus australis,1,O,Yes,Yes,No,No,No,No,3
Apristurus breviventralis,1,O,Yes,Yes,No,No,No,No,3


In [7]:
states<-df$Consensus
names(states)=str_replace((rownames(df)), " ", "_")
states2<-states[!names(states) %in% setdiff(names(states), phy$tip.label)]

In [8]:
phy2.0<-drop.tip(phy, setdiff(phy$tip.label, (names(states))))

### Computing sampling fraction

In [9]:
### Computing sampling fraction

In [10]:
samp_frac<-c(
length(states2[states2=="1"])/length(df$Consensus[df$Consensus=="1"]),
length(states2[states2=="2"])/length(df$Consensus[df$Consensus=="2"]),
length(states2[states2=="3"])/length(df$Consensus[df$Consensus=="3"]))

### Naming states according to Secsse

In [11]:
### Naming states according to Secsse

In [12]:
states3<-as.data.frame(cbind(names(states2), as.factor(states2)))
colnames(states3)<-c("species","states")
rownames(states3)<-NULL
traits <- sortingtraits(states3, phy2.0)

## II Running the analysis

In [13]:
## II Running the analysis

### Initialisation

In [14]:
### Initialisation

In [15]:
startingpoint <- bd_ML(brts = ape::branching.times(phy2.0))
intGuessLambda <- startingpoint$lambda0
intGuessMu <- startingpoint$mu0
#COND=conditioning on the state of the root
cond<-"proper_cond"
#root_state_weight
root_state_weight<-"proper_weights"
#SAMPLING_FRACTION
#TOL
tol = c(1e-04, 1e-05, 1e-07)

You are optimizing lambda0 mu0 
You are fixing lambda1 mu1 
Optimizing the likelihood - this may take a while. 
The loglikelihood for the initial parameter values is -2042.37935685298.

Maximum likelihood parameter estimates: lambda0: 0.028186, mu0: 0.009724, lambda1: 0.000000, mu1: 0.000000: 
Maximum loglikelihood: -1780.702572


### Constant rate model

In [16]:
### Constant rate model

In [17]:
idparslist <- id_paramPos(traits, num_concealed_states = 3)
idparslist[[1]][] <- 1 #same speciation rate
idparslist[[2]][] <- 2 #same extinction rate
diag(idparslist[[3]]) <- NA#remove same state transition rate (diagonal)
#all transition rates equal
idparslist[[3]][1,c(2,4,7)] <- 3
idparslist[[3]][1,c(3,5,6,8,9)] <- 0
idparslist[[3]][2,c(1,3,5,8)] <- 3
idparslist[[3]][2,c(4,6,7,9)] <- 0
idparslist[[3]][3,c(2,6,9)] <- 3
idparslist[[3]][3,c(1,4,5,7,8)] <- 0
idparslist[[3]][4,c(1,5,7)] <- 3
idparslist[[3]][4,c(2,3,6,8,9)] <- 0
idparslist[[3]][5,c(2,4,6,8)] <- 3
idparslist[[3]][5,c(1,3,7,9)] <- 0
idparslist[[3]][6,c(3,5,9)] <- 3
idparslist[[3]][6,c(1,2,4,7,8)] <- 0
idparslist[[3]][7,c(1,4,8)] <- 3
idparslist[[3]][7,c(2,3,5,6,9)] <- 0
idparslist[[3]][8,c(2,5,7,9)] <- 3
idparslist[[3]][8,c(1,3,4,6)] <- 0
idparslist[[3]][9,c(3,6,8)] <- 3
idparslist[[3]][9,c(1,2,4,5,7)] <- 0


In [18]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(intGuessLambda,intGuessMu,0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:3)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CR_sp<-secsse_ml(phy2.0,traits, num_concealed_states=3, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2216.778 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0097240310612135 0.25 -2216.77796730219 initial 
2 0.0295954409554525 0.0102102326142742 0.225728155339806 -2213.75116829953 expand 
3 0.0295954409554525 0.0102102326142742 0.225728155339806 -2213.75116829953 reflect 
4 0.0328988958094577 0.0100481134126242 0.210063897763578 -2212.53477906308 expand 
5 0.0333725491948891 0.0103724038666223 0.172600619195047 -2209.07199856638 expand 
6 0.0357473471514961 0.011509062160642 0.144259818731118 -2206.67140841561 expand 
7 0.0357473471514961 0.011509062160642 0.144259818731118 -2206.67140841561 reflect 
8 0.0418126390331958 0.0128655658212533 0.0414757103574704 -2163.67628241352 expand 
9 0.0418126390331958 0.0128655658212533 0.0414757103574704 -2163.67628241352 reflect 
10 0.0420089008810875 0.0139

90 0.0382016303837857 0.0198904641825938 0.00390665540245412 -2024.24407159865 reflect 
91 0.0382225148536897 0.0197712050893399 0.00389059587517774 -2024.24255698974 contract inside 
92 0.0382225148536897 0.0197712050893399 0.00389059587517774 -2024.24255698974 contract outside 
93 0.0382225148536897 0.0197712050893399 0.00389059587517774 -2024.24255698974 contract inside 
94 0.0382225148536897 0.0197712050893399 0.00389059587517774 -2024.24255698974 contract inside 
95 0.0382225148536897 0.0197712050893399 0.00389059587517774 -2024.24255698974 reflect 
96 0.0382970599971472 0.0199030833257721 0.00389917883482851 -2024.24243113163 contract inside 
97 0.0382970599971472 0.0199030833257721 0.00389917883482851 -2024.24243113163 reflect 
98 0.0382650526413673 0.0198828243053437 0.00389499935277815 -2024.24239186399 contract inside 
99 0.0382650526413673 0.0198828243053437 0.00389499935277815 -2024.24239186399 contract outside 
100 0.0382451290528258 0.0198195445380547 0.0038944183914952 -

### CTD2 model (Two conceal states)

In [19]:
### CTD2 model (Two conceal states)

In [20]:
idparslist <- id_paramPos(traits, num_concealed_states = 2)
idparslist[[1]][1:3] <- 1 #Speciation varies only between concealed states
idparslist[[1]][4:6] <- 2 
idparslist[[2]][] <- 3 #same extinction rate

diag(idparslist[[3]]) <- NA#remove same state transition rate (diagonal)
#all transition rates equal
idparslist[[3]][1,c(2,4)] <- 4
idparslist[[3]][1,c(3,5,6)] <- 0
idparslist[[3]][2,c(1,3,5)] <- 4
idparslist[[3]][2,c(4,6)] <- 0
idparslist[[3]][3,c(2,6)] <- 4
idparslist[[3]][3,c(1,4,5)] <- 0
idparslist[[3]][4,c(1,5)] <- 4
idparslist[[3]][4,c(2,3,6)] <- 0
idparslist[[3]][5,c(2,4,6)] <- 4
idparslist[[3]][5,c(1,3)] <- 0
idparslist[[3]][6,c(3,5)] <- 4
idparslist[[3]][6,c(1,2,4)] <- 0

In [21]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda, 2),intGuessMu,0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:4)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CTD2_sp<-secsse_ml(phy2.0,traits, num_concealed_states=2, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2216.778 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2216.7779673023 initial 
2 0.0292427522080496 0.0292427522080496 0.0100886383346542 0.225728155339806 -2214.09974857915 expand 
3 0.0292427522080496 0.0292427522080496 0.0100886383346542 0.225728155339806 -2214.09974857915 reflect 
4 0.0309643959176843 0.0309643959176843 0.0097695925748247 0.222760290556901 -2212.87251675186 expand 
5 0.0330579653042333 0.0291656337512935 0.00980376640844103 0.203097081596188 -2211.36912875164 expand 
6 0.032822153051782 0.0327334045656857 0.00986357618350512 0.170166545981173 -2208.72574776681 expand 
7 0.0357461849186743 0.0327424173879456 0.0108046131842184 0.144516448882751 -2205.41661650349 expand 
8 0.0357461849186743 0.0327424173879456 0.0108046131842184 0.1445

74 0.0544735268307751 0.0262795032579025 0.012963378095095 0.00332776445597029 -2018.56166491704 reflect 
75 0.0545126146946478 0.0259798410873817 0.0130311805242951 0.00326255113513351 -2018.52551452887 expand 
76 0.0543370092294817 0.0262206545291361 0.0130032462992593 0.00340670790489563 -2018.51009930529 expand 
77 0.0541324743345731 0.0264589953914574 0.0129865135303615 0.0033225013906766 -2018.46173652342 expand 
78 0.0541324743345731 0.0264589953914574 0.0129865135303615 0.0033225013906766 -2018.46173652342 reflect 
79 0.0539815115404131 0.0260733092152773 0.0131092132808568 0.00339287340192543 -2018.31132424243 expand 
80 0.0539815115404131 0.0260733092152773 0.0131092132808568 0.00339287340192543 -2018.31132424243 reflect 
81 0.0534844972862586 0.0264620415496594 0.0131094071079666 0.00340387564220892 -2018.19720674678 expand 
82 0.0534844972862586 0.0264620415496594 0.0131094071079666 0.00340387564220892 -2018.19720674678 reflect 
83 0.0528143092758624 0.0262682070489455 0.01

149 0.0438481144359817 0.0232252405062236 0.0158050857251716 0.00374051084811477 -2013.73844545154 reflect 
150 0.0438653722393567 0.023208983659931 0.0158054534346102 0.00374370668902747 -2013.73843121145 expand 
151 0.043838367847881 0.0232135777093346 0.0158095080982895 0.00374368149466002 -2013.73837357645 expand 
152 0.0438601150461449 0.0232288128390342 0.0158012489129191 0.00374277601711171 -2013.73830289047 expand 
153 0.0438641341454028 0.0232119902544444 0.0158046887160195 0.00373802293606931 -2013.73825526215 expand 
154 0.0438747631973337 0.0231970426802396 0.0158055028956819 0.0037451186539725 -2013.73816225208 expand 
155 0.0438472904037976 0.0232206000094907 0.0158048045727524 0.00373978593734436 -2013.73787805771 expand 
156 0.0438472904037976 0.0232206000094907 0.0158048045727524 0.00373978593734436 -2013.73787805771 reflect 
157 0.0438829987945051 0.0231763342834352 0.015807710194615 0.0037360213240839 -2013.73775846571 expand 
158 0.0438891090527877 0.023183235554843

225 0.0453328314548953 0.0161316367096225 0.000842848004875779 0.00343823788517887 -2010.71118922776 reflect 
226 0.0452300237658314 0.0157592258392536 0.000308605727211623 0.00332658091208486 -2010.69538141698 reflect 
227 0.0452300237658314 0.0157592258392536 0.000308605727211623 0.00332658091208486 -2010.69538141698 reflect 
228 0.0452300237658314 0.0157592258392536 0.000308605727211623 0.00332658091208486 -2010.69538141698 reflect 
229 0.0453277813522106 0.0159382833612957 0.000617389427423304 0.00345805397666289 -2010.65002498832 expand 
230 0.0453277813522106 0.0159382833612957 0.000617389427423304 0.00345805397666289 -2010.65002498832 reflect 
231 0.0451285365006342 0.0161215244634382 4.4021877517344e-05 0.00332204494770358 -2010.64028461764 reflect 
232 0.0451285365006342 0.0161215244634382 4.4021877517344e-05 0.00332204494770358 -2010.64028461764 reflect 
233 0.0451285365006342 0.0161215244634382 4.4021877517344e-05 0.00332204494770358 -2010.64028461764 reflect 
234 0.04531656

299 0.0450659697409214 0.0159784660558819 1.01592561839223e-05 0.00359506459981462 -2010.42671376351 expand 
300 0.0450397757323807 0.0160304865565765 3.02450838708211e-06 0.00359426704086476 -2010.42626157139 reflect 
301 0.0450327641922685 0.0159790862764544 4.30695465234688e-06 0.00356199785851421 -2010.42028677173 expand 
302 0.0449381437569888 0.0161229360911255 9.20762448357864e-06 0.00357933841333654 -2010.41712932852 expand 
303 0.0449381437569888 0.0161229360911255 9.20762448357864e-06 0.00357933841333654 -2010.41712932852 reflect 
304 0.0449507822803814 0.0160896995984174 3.02829618965928e-06 0.00357958809206919 -2010.41360045609 reflect 
305 0.0448788382287123 0.0160856763880206 1.37351909990917e-05 0.00356243627006839 -2010.39795970549 expand 
306 0.0448788382287123 0.0160856763880206 1.37351909990917e-05 0.00356243627006839 -2010.39795970549 reflect 
307 0.0448788382287123 0.0160856763880206 1.37351909990917e-05 0.00356243627006839 -2010.39795970549 reflect 
308 0.04473711

...

448 0.0377146756638769 0.0123303046905216 4.53857050363521e-06 0.00383087869540802 -2008.52007867673 expand 
449 0.0377146756638769 0.0123303046905216 4.53857050363521e-06 0.00383087869540802 -2008.52007867673 contract inside 
450 0.0377146756638769 0.0123303046905216 4.53857050363521e-06 0.00383087869540802 -2008.52007867673 reflect 
451 0.0377146756638769 0.0123303046905216 4.53857050363521e-06 0.00383087869540802 -2008.52007867673 reflect 
452 0.0375990954152024 0.0123150858006446 2.51229025363673e-06 0.00381823812628393 -2008.51777049602 reflect 
453 0.0375990954152024 0.0123150858006446 2.51229025363673e-06 0.00381823812628393 -2008.51777049602 reflect 
454 0.0375990954152024 0.0123150858006446 2.51229025363673e-06 0.00381823812628393 -2008.51777049602 contract inside 
455 0.0375990954152024 0.0123150858006446 2.51229025363673e-06 0.00381823812628393 -2008.51777049602 contract inside 
456 0.0375990954152024 0.0123150858006446 2.51229025363673e-06 0.00381823812628393 -2008.51777049

521 0.0375694076881562 0.012394766508974 1.50746638182405e-09 0.00381652443082556 -2008.51615613705 reflect 
522 0.0375694076881562 0.012394766508974 1.50746638182405e-09 0.00381652443082556 -2008.51615613705 contract inside 
523 0.0375694076881562 0.012394766508974 1.50746638182405e-09 0.00381652443082556 -2008.51615613705 reflect 
524 0.0375694076881562 0.012394766508974 1.50746638182405e-09 0.00381652443082556 -2008.51615613705 reflect 
525 0.037571647314597 0.012396526038306 4.72332460970458e-09 0.00381610796804292 -2008.51615582789 reflect 
526 0.037571647314597 0.012396526038306 4.72332460970458e-09 0.00381610796804292 -2008.51615582789 contract inside 
527 0.0375717769150106 0.0123959362349427 6.52733729026576e-10 0.00381632250123829 -2008.5161555519 reflect 
528 0.0375717769150106 0.0123959362349427 6.52733729026576e-10 0.00381632250123829 -2008.5161555519 reflect 
529 0.0375717769150106 0.0123959362349427 6.52733729026576e-10 0.00381632250123829 -2008.5161555519 contract insid

594 0.0375703979524812 0.0123971295812001 1.1348194101348e-11 0.00381609039270822 -2008.51615494606 reflect 
595 0.0375703979524812 0.0123971295812001 1.1348194101348e-11 0.00381609039270822 -2008.51615494606 reflect 
596 0.0375703979524812 0.0123971295812001 1.1348194101348e-11 0.00381609039270822 -2008.51615494606 contract outside 
597 0.0375703486252078 0.0123970950620546 1.35200181232816e-11 0.00381609602994266 -2008.51615494502 expand 
598 0.0375703486252078 0.0123970950620546 1.35200181232816e-11 0.00381609602994266 -2008.51615494502 contract inside 
599 0.0375703024184455 0.0123971284975326 5.55578016939715e-12 0.00381608893485299 -2008.5161549441 reflect 
600 0.0375704787134144 0.0123971120099246 2.32477345486542e-13 0.00381609347706127 -2008.51615494349 expand 
601 0.0375704787134144 0.0123971120099246 2.32477345486542e-13 0.00381609347706127 -2008.51615494349 contract inside 
602 0.0375703171830042 0.0123971043054627 1.12694132653255e-12 0.00381609175432035 -2008.51615494007 

668 0.0375696830312119 0.0123957825494126 5.4462548861874e-11 0.00381591559549058 -2008.5161546537 reflect 
669 0.037569384157026 0.012395611677513 6.09876200778872e-11 0.00381579485357691 -2008.51615463636 expand 
670 0.037569384157026 0.012395611677513 6.09876200778872e-11 0.00381579485357691 -2008.51615463636 reflect 
671 0.037570116046191 0.0123958475537701 8.93970897256525e-11 0.00381578192552005 -2008.51615461695 expand 
672 0.0375701046863926 0.0123961779228742 1.11817842693522e-10 0.00381574625123561 -2008.5161546038 expand 
673 0.0375701046863926 0.0123961779228742 1.11817842693522e-10 0.00381574625123561 -2008.5161546038 reflect 
674 0.0375701046863926 0.0123961779228742 1.11817842693522e-10 0.00381574625123561 -2008.5161546038 reflect 
675 0.0375701046863926 0.0123961779228742 1.11817842693522e-10 0.00381574625123561 -2008.5161546038 reflect 
676 0.0375701046863926 0.0123961779228742 1.11817842693522e-10 0.00381574625123561 -2008.5161546038 contract outside 
677 0.0375700438

742 0.0375699217575685 0.0123961488029336 4.36425247354865e-14 0.00381575270616444 -2008.51615458188 reflect 
743 0.0375699217575685 0.0123961488029336 4.36425247354865e-14 0.00381575270616444 -2008.51615458188 reflect 
744 0.0375699217575685 0.0123961488029336 4.36425247354865e-14 0.00381575270616444 -2008.51615458188 reflect 
745 0.0375699217575685 0.0123961488029336 4.36425247354865e-14 0.00381575270616444 -2008.51615458188 contract inside 
746 0.0375699217575685 0.0123961488029336 4.36425247354865e-14 0.00381575270616444 -2008.51615458188 reflect 
747 0.0375699146946987 0.0123961419845082 3.37444731364728e-14 0.00381575136894672 -2008.51615458188 reflect 
748 0.0375699163841078 0.0123961443024597 3.4828223935284e-15 0.00381575259566186 -2008.51615458187 reflect 
749 0.0375699163841078 0.0123961443024597 3.4828223935284e-15 0.00381575259566186 -2008.51615458187 contract inside 
750 0.0375699163841078 0.0123961443024597 3.4828223935284e-15 0.00381575259566186 -2008.51615458187 contra

### CTD3 model (Three conceal states)

In [22]:
### CTD3 model (Three conceal states)

In [23]:
idparslist <- id_paramPos(traits, num_concealed_states = 3)
idparslist[[1]][1:3] <- 1 #Speciation varies only between concealed states
idparslist[[1]][4:6] <- 2 
idparslist[[1]][7:9] <- 3 
idparslist[[2]][] <- 4 #same extinction rate

diag(idparslist[[3]]) <- NA#remove same state transition rate (diagonal)
#all transition rates equal
idparslist[[3]][1,c(2,4,7)] <- 5
idparslist[[3]][1,c(3,5,6,8,9)] <- 0
idparslist[[3]][2,c(1,3,5,8)] <- 5
idparslist[[3]][2,c(4,6,7,9)] <- 0
idparslist[[3]][3,c(2,6,9)] <- 5
idparslist[[3]][3,c(1,4,5,7,8)] <- 0
idparslist[[3]][4,c(1,5,7)] <- 5
idparslist[[3]][4,c(2,3,6,8,9)] <- 0
idparslist[[3]][5,c(2,4,6,8)] <- 5
idparslist[[3]][5,c(1,3,7,9)] <- 0
idparslist[[3]][6,c(3,5,9)] <- 5
idparslist[[3]][6,c(1,2,4,7,8)] <- 0
idparslist[[3]][7,c(1,4,8)] <- 5
idparslist[[3]][7,c(2,3,5,6,9)] <- 0
idparslist[[3]][8,c(2,5,7,9)] <- 5
idparslist[[3]][8,c(1,3,4,6)] <- 0
idparslist[[3]][9,c(3,6,8)] <- 5
idparslist[[3]][9,c(1,2,4,5,7)] <- 0

In [24]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda, 3),intGuessMu,0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:5)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CTD3_sp<-secsse_ml(phy2.0,traits, num_concealed_states=3, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2216.778 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2216.77796730219 initial 
2 0.0290312549170369 0.0290312549170369 0.0290312549170369 0.0100156958161729 0.225728155339806 -2214.33256959003 expand 
3 0.0290312549170369 0.0290312549170369 0.0290312549170369 0.0100156958161729 0.225728155339806 -2214.33256959003 reflect 
4 0.0290312549170369 0.0290312549170369 0.0290312549170369 0.0100156958161729 0.225728155339806 -2214.33256959003 reflect 
5 0.0308410939666489 0.0271760673764524 0.0308410939666489 0.00965872124701668 0.221553525814692 -2213.38271466041 expand 
6 0.0287642721124852 0.029384588676968 0.0324406613687387 0.00961953941366409 0.205098736961071 -2211.83851291848 expand 
7 0.0319390386890017 0.0301050033182086 0.0313156405

63 0.0270168152070585 0.0334570218669907 0.0474102828670408 0.0105609552386452 0.00375793595909507 -2019.34186901142 reflect 
64 0.0220479942522747 0.0345384193889251 0.0496720357196337 0.0109706732047878 0.00301209616117465 -2016.75362640122 expand 
65 0.0220479942522747 0.0345384193889251 0.0496720357196337 0.0109706732047878 0.00301209616117465 -2016.75362640122 reflect 
66 0.0221400841231752 0.0343637642630471 0.0504124476294259 0.0106610063922062 0.00350835534558521 -2016.66436952679 reflect 
67 0.0193616118333013 0.0348656946231874 0.0514346532366557 0.0110997587406781 0.0033650194348189 -2016.51915707372 reflect 
68 0.0193616118333013 0.0348656946231874 0.0514346532366557 0.0110997587406781 0.0033650194348189 -2016.51915707372 reflect 
69 0.0193616118333013 0.0348656946231874 0.0514346532366557 0.0110997587406781 0.0033650194348189 -2016.51915707372 contract inside 
70 0.0196683616274368 0.0348758402874466 0.0513538792801647 0.0109881941465041 0.00322792138747879 -2016.510026301

128 0.0214386570135179 0.0344634692006471 0.0453025511256352 0.0135595348629498 0.00367575748296548 -2015.46915610296 contract inside 
129 0.0216293935441769 0.0343790673039961 0.0448675220413303 0.0137656408913847 0.00366692516185831 -2015.44500444929 expand 
130 0.0216293935441769 0.0343790673039961 0.0448675220413303 0.0137656408913847 0.00366692516185831 -2015.44500444929 reflect 
131 0.0216293935441769 0.0343790673039961 0.0448675220413303 0.0137656408913847 0.00366692516185831 -2015.44500444929 reflect 
132 0.0216293935441769 0.0343790673039961 0.0448675220413303 0.0137656408913847 0.00366692516185831 -2015.44500444929 reflect 
133 0.0216293935441769 0.0343790673039961 0.0448675220413303 0.0137656408913847 0.00366692516185831 -2015.44500444929 reflect 
134 0.0216293935441769 0.0343790673039961 0.0448675220413303 0.0137656408913847 0.00366692516185831 -2015.44500444929 reflect 
135 0.0216293935441769 0.0343790673039961 0.0448675220413303 0.0137656408913847 0.00366692516185831 -201

192 0.0177032351415004 0.0288080565235377 0.048418610471615 0.0188393768076514 0.00392191951659064 -2012.36101389012 reflect 
193 0.0177032351415004 0.0288080565235377 0.048418610471615 0.0188393768076514 0.00392191951659064 -2012.36101389012 reflect 
194 0.0177032351415004 0.0288080565235377 0.048418610471615 0.0188393768076514 0.00392191951659064 -2012.36101389012 contract inside 
195 0.0177032351415004 0.0288080565235377 0.048418610471615 0.0188393768076514 0.00392191951659064 -2012.36101389012 reflect 
196 0.0177032351415004 0.0288080565235377 0.048418610471615 0.0188393768076514 0.00392191951659064 -2012.36101389012 reflect 
197 0.0178875623868766 0.0287699030063162 0.0483735111657593 0.0188023140028361 0.00393013407837113 -2012.35726190031 expand 
198 0.0178875623868766 0.0287699030063162 0.0483735111657593 0.0188023140028361 0.00393013407837113 -2012.35726190031 reflect 
199 0.0178875623868766 0.0287699030063162 0.0483735111657593 0.0188023140028361 0.00393013407837113 -2012.357

257 0.0233588193601624 0.0268881108055295 0.0480418228549649 0.0178867430049452 0.00391153692945577 -2012.27916063156 reflect 
258 0.023378452748845 0.0268858264716335 0.0480160913333 0.0178925243202646 0.00390917005590975 -2012.27914854094 contract outside 
259 0.0233555832838611 0.0268809056190106 0.0480280840342189 0.0179022907956128 0.00390852950616767 -2012.27913954125 contract inside 
260 0.0233555832838611 0.0268809056190106 0.0480280840342189 0.0179022907956128 0.00390852950616767 -2012.27913954125 contract inside 
261 0.0233310323655712 0.0269153674790633 0.0480274064915077 0.0178840613291821 0.00390784380117998 -2012.2791355821 contract inside 
262 0.0233101874454604 0.0269077808047663 0.0480305960752348 0.0178998424260546 0.0039069192646633 -2012.27912973006 contract inside 
263 0.0233554158899741 0.0268912865117308 0.0480336833022061 0.0178900086892058 0.00390961293537177 -2012.2791283947 contract inside 
264 0.0233554158899741 0.0268912865117308 0.0480336833022061 0.017890

...

577 0.0120898944285581 0.0221290662443419 0.0488165382155101 3.71494624464327e-06 0.00350456268742178 -2008.66699863766 reflect 
578 0.0120898944285581 0.0221290662443419 0.0488165382155101 3.71494624464327e-06 0.00350456268742178 -2008.66699863766 contract inside 
579 0.0120898944285581 0.0221290662443419 0.0488165382155101 3.71494624464327e-06 0.00350456268742178 -2008.66699863766 contract inside 
580 0.0120898944285581 0.0221290662443419 0.0488165382155101 3.71494624464327e-06 0.00350456268742178 -2008.66699863766 contract inside 
581 0.0120977446217962 0.0220886730794974 0.0488429735417026 1.54948030215655e-06 0.0035027295612312 -2008.66633693176 reflect 
582 0.0120977446217962 0.0220886730794974 0.0488429735417026 1.54948030215655e-06 0.0035027295612312 -2008.66633693176 contract inside 
583 0.0120977446217962 0.0220886730794974 0.0488429735417026 1.54948030215655e-06 0.0035027295612312 -2008.66633693176 reflect 
584 0.0120977446217962 0.0220886730794974 0.0488429735417026 1.54948

640 0.0120236817670451 0.0220604301800384 0.0488893397829779 1.16738195127504e-08 0.00352951545368345 -2008.66365802665 contract inside 
641 0.0120050381558999 0.0220645818520643 0.0488904703691198 2.63726640754634e-08 0.00353049287143489 -2008.66363295364 reflect 
642 0.0120050381558999 0.0220645818520643 0.0488904703691198 2.63726640754634e-08 0.00353049287143489 -2008.66363295364 contract inside 
643 0.0120050381558999 0.0220645818520643 0.0488904703691198 2.63726640754634e-08 0.00353049287143489 -2008.66363295364 contract inside 
644 0.0120050381558999 0.0220645818520643 0.0488904703691198 2.63726640754634e-08 0.00353049287143489 -2008.66363295364 contract inside 
645 0.0120050381558999 0.0220645818520643 0.0488904703691198 2.63726640754634e-08 0.00353049287143489 -2008.66363295364 contract outside 
646 0.0120050381558999 0.0220645818520643 0.0488904703691198 2.63726640754634e-08 0.00353049287143489 -2008.66363295364 contract inside 
647 0.0120050381558999 0.0220645818520643 0.0488

702 0.0120048502566825 0.0220650591065859 0.0488894469681688 1.95784563759672e-11 0.00352882489358974 -2008.66361664792 contract inside 
703 0.0120048502566825 0.0220650591065859 0.0488894469681688 1.95784563759672e-11 0.00352882489358974 -2008.66361664792 contract inside 
704 0.0120042063735214 0.0220654064291584 0.0488893554240273 8.28053851144034e-12 0.00352890112424145 -2008.66361663159 reflect 
705 0.0120042063735214 0.0220654064291584 0.0488893554240273 8.28053851144034e-12 0.00352890112424145 -2008.66361663159 contract inside 
706 0.0120042063735214 0.0220654064291584 0.0488893554240273 8.28053851144034e-12 0.00352890112424145 -2008.66361663159 contract inside 
707 0.0120046854849513 0.0220652650300872 0.04888928809382 9.38748565953163e-12 0.00352872582102762 -2008.66361660572 reflect 
708 0.0120046854849513 0.0220652650300872 0.04888928809382 9.38748565953163e-12 0.00352872582102762 -2008.66361660572 contract inside 
709 0.0120046854849513 0.0220652650300872 0.04888928809382 9.

764 0.0120041015915817 0.022065541373169 0.0488892125523754 2.88693313239327e-13 0.00352874622086555 -2008.66361657438 contract inside 
765 0.0120041015915817 0.022065541373169 0.0488892125523754 2.88693313239327e-13 0.00352874622086555 -2008.66361657438 contract outside 
766 0.0120041015915817 0.022065541373169 0.0488892125523754 2.88693313239327e-13 0.00352874622086555 -2008.66361657438 contract inside 
767 0.0120041015915817 0.022065541373169 0.0488892125523754 2.88693313239327e-13 0.00352874622086555 -2008.66361657438 contract outside 
768 0.0120041015915817 0.022065541373169 0.0488892125523754 2.88693313239327e-13 0.00352874622086555 -2008.66361657438 contract inside 
769 0.0120041258744567 0.0220655642754885 0.0488891880598583 1.48807949476144e-12 0.00352873924963702 -2008.66361657437 contract inside 
770 0.0120041469684664 0.0220655459045851 0.0488892004644473 1.37137431064295e-12 0.00352874693784168 -2008.66361657434 contract outside 
771 0.0120041450120242 0.0220655267273187 0

827 0.012004174300918 0.0220655736287822 0.0488891703939119 8.9952266021716e-15 0.00352873653024932 -2008.6636165736 reflect 
828 0.012004174300918 0.0220655736287822 0.0488891703939119 8.9952266021716e-15 0.00352873653024932 -2008.6636165736 contract inside 
829 0.012004174300918 0.0220655736287822 0.0488891703939119 8.9952266021716e-15 0.00352873653024932 -2008.6636165736 reflect 
830 0.012004174300918 0.0220655736287822 0.0488891703939119 8.9952266021716e-15 0.00352873653024932 -2008.6636165736 reflect 
831 0.0120041714907229 0.0220655741560167 0.0488891711331231 7.67199846203065e-15 0.00352873785195228 -2008.66361657359 expand 
832 0.0120041714907229 0.0220655741560167 0.0488891711331231 7.67199846203065e-15 0.00352873785195228 -2008.66361657359 reflect 
833 0.0120041714907229 0.0220655741560167 0.0488891711331231 7.67199846203065e-15 0.00352873785195228 -2008.66361657359 reflect 
834 0.0120041714907229 0.0220655741560167 0.0488891711331231 7.67199846203065e-15 0.00352873785195228 

### CTD4 model (Four conceal states)

In [25]:
### CTD4 model (Four conceal states)

In [26]:
idparslist <- id_paramPos(traits, num_concealed_states = 4)
idparslist[[1]][1:3] <- 1 #Speciation varies only between concealed states
idparslist[[1]][4:6] <- 2 
idparslist[[1]][7:9] <- 3  
idparslist[[1]][10:12] <- 4 
idparslist[[2]][] <- 5 #same extinction rate

diag(idparslist[[3]]) <- NA#remove same state transition rate (diagonal)
#all transition rates equal
idparslist[[3]][1,c(2,4,7,10)] <- 6
idparslist[[3]][1,c(3,5,6,8,9,11,12)] <- 0
idparslist[[3]][2,c(1,3,5,8,11)] <- 6
idparslist[[3]][2,c(4,6,7,9,10,12)] <- 0
idparslist[[3]][3,c(2,6,9,12)] <- 6
idparslist[[3]][3,c(1,4,5,7,8,10,11)] <- 0
idparslist[[3]][4,c(1,5,7,10)] <- 6
idparslist[[3]][4,c(2,3,6,8,9,11,12)] <- 0
idparslist[[3]][5,c(2,4,6,8,11)] <- 6
idparslist[[3]][5,c(1,3,7,9,10,12)] <- 0
idparslist[[3]][6,c(3,5,9,12)] <- 6
idparslist[[3]][6,c(1,2,4,7,8,10,11)] <- 0
idparslist[[3]][7,c(1,4,8,10)] <- 6
idparslist[[3]][7,c(2,3,5,6,9,11,12)] <- 0
idparslist[[3]][8,c(2,5,7,9,11)] <- 6
idparslist[[3]][8,c(1,3,4,6,10,12)] <- 0
idparslist[[3]][9,c(3,6,8,12)] <- 6
idparslist[[3]][9,c(1,2,4,5,7,10,11)] <- 0
idparslist[[3]][10,c(1,4,7,11)] <- 6
idparslist[[3]][10,c(2,3,5,6,8,9,12)] <- 0
idparslist[[3]][11,c(2,5,8,10,12)] <- 6
idparslist[[3]][11,c(1,3,4,6,7,9)] <- 0
idparslist[[3]][12,c(3,6,9,11)] <- 6
idparslist[[3]][12,c(1,2,4,5,7,8,10)] <- 0
idparslist[[3]]

,1A,2A,3A,1B,2B,3B,1C,2C,3C,1D,2D,3D
1A,NA,6,0,6,0,0,6,0,0,6,0,0
2A,6,NA,6,0,6,0,0,6,0,0,6,0
3A,0,6,NA,0,0,6,0,0,6,0,0,6
1B,6,0,0,NA,6,0,6,0,0,6,0,0
2B,0,6,0,6,NA,6,0,6,0,0,6,0
3B,0,0,6,0,6,NA,0,0,6,0,0,6
1C,6,0,0,6,0,0,NA,6,0,6,0,0
2C,0,6,0,0,6,0,6,NA,6,0,6,0
3C,0,0,6,0,0,6,0,6,NA,0,0,6
1D,6,0,0,6,0,0,6,0,0,NA,6,0


In [27]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda, 4),intGuessMu,0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:6)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CTD4_sp<-secsse_ml(phy2.0,traits, num_concealed_states=4, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2216.778 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2216.77796730212 initial 
2 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.00996707332298065 0.225728155339806 -2214.49779680596 expand 
3 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.00996707332298065 0.225728155339806 -2214.49779680596 reflect 
4 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.00996707332298065 0.225728155339806 -2214.49779680596 reflect 
5 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.00996707332298065 0.225728155339806 -2214.49779680596 reflect 
6 0.0306942422390158 0.0306942422390158 0.0271706986581883 0.02904691840218

56 0.0402769997565008 0.0336680952140251 0.0344245252572025 0.0387940776543696 0.00933800550199965 0.00354983915036688 -2031.12569305079 contract inside 
57 0.0402377207569632 0.0335762537123103 0.0343204085720717 0.038313501342142 0.00945905697833079 0.00423516372564539 -2030.83819294823 reflect 
58 0.0402377207569632 0.0335762537123103 0.0343204085720717 0.038313501342142 0.00945905697833079 0.00423516372564539 -2030.83819294823 contract inside 
59 0.0402377207569632 0.0335762537123103 0.0343204085720717 0.038313501342142 0.00945905697833079 0.00423516372564539 -2030.83819294823 reflect 
60 0.0405041808138748 0.0328954832105349 0.0346053779132239 0.037632665317839 0.00949045255921954 0.00341916658179752 -2029.98183722656 expand 
61 0.0405041808138748 0.0328954832105349 0.0346053779132239 0.037632665317839 0.00949045255921954 0.00341916658179752 -2029.98183722656 contract inside 
62 0.0405041808138748 0.0328954832105349 0.0346053779132239 0.037632665317839 0.00949045255921954 0.003419

112 0.0429688358713466 0.0173347613764042 0.0356527047866661 0.000229222589367252 0.0172380447324674 0.00403170076623717 -2012.14593390201 contract inside 
113 0.0429688358713466 0.0173347613764042 0.0356527047866661 0.000229222589367252 0.0172380447324674 0.00403170076623717 -2012.14593390201 reflect 
114 0.0429688358713466 0.0173347613764042 0.0356527047866661 0.000229222589367252 0.0172380447324674 0.00403170076623717 -2012.14593390201 contract inside 
115 0.0430357673915916 0.017280989569849 0.035673465806358 0.000455482155547753 0.0171763201690221 0.00388406096401337 -2012.13172554176 reflect 
116 0.0430357673915916 0.017280989569849 0.035673465806358 0.000455482155547753 0.0171763201690221 0.00388406096401337 -2012.13172554176 contract inside 
117 0.0430357673915916 0.017280989569849 0.035673465806358 0.000455482155547753 0.0171763201690221 0.00388406096401337 -2012.13172554176 contract inside 
118 0.0430357673915916 0.017280989569849 0.035673465806358 0.000455482155547753 0.0171

168 0.0481041895062735 0.00876007122819403 0.0358095853776519 0.000104626759137493 0.0164088662185698 0.00447396880500998 -2009.30894290357 contract inside 
169 0.049538761350399 0.00653734968806553 0.0358505545340675 0.000228738786199817 0.0161343241426579 0.00439610315144919 -2009.25331861595 expand 
170 0.049538761350399 0.00653734968806553 0.0358505545340675 0.000228738786199817 0.0161343241426579 0.00439610315144919 -2009.25331861595 contract inside 
171 0.049538761350399 0.00653734968806553 0.0358505545340675 0.000228738786199817 0.0161343241426579 0.00439610315144919 -2009.25331861595 contract inside 
172 0.0484435326650874 0.00824562941551006 0.0358224604710712 1.4776486237411e-05 0.0163644633885865 0.00437139661998372 -2009.23290695707 expand 
173 0.0484828166313673 0.008209195778628 0.035812852936028 0.000111246412949091 0.01634076655765 0.00436903785187466 -2009.23277708908 reflect 
174 0.0484828166313673 0.008209195778628 0.035812852936028 0.000111246412949091 0.01634076655

222 0.0490539710165982 0.00732676999474425 0.0358098113312611 1.48623895797954e-06 0.0162635218602849 0.00437946515530176 -2009.19626050704 reflect 
223 0.0489925348813855 0.00744417526064926 0.0357924419254552 2.93870388589044e-06 0.0162737182380077 0.00441815278703683 -2009.19603012549 expand 
224 0.0489925348813855 0.00744417526064926 0.0357924419254552 2.93870388589044e-06 0.0162737182380077 0.00441815278703683 -2009.19603012549 contract inside 
225 0.0489925348813855 0.00744417526064926 0.0357924419254552 2.93870388589044e-06 0.0162737182380077 0.00441815278703683 -2009.19603012549 contract inside 
226 0.0489925348813855 0.00744417526064926 0.0357924419254552 2.93870388589044e-06 0.0162737182380077 0.00441815278703683 -2009.19603012549 contract inside 
227 0.0489925348813855 0.00744417526064926 0.0357924419254552 2.93870388589044e-06 0.0162737182380077 0.00441815278703683 -2009.19603012549 reflect 
228 0.0489925348813855 0.00744417526064926 0.0357924419254552 2.93870388589044e-06 

...

880 0.0503922489278784 0.00387047186947708 0.0314865135279323 0.000158689798965779 0.018566334390642 0.00419893194124576 -2007.61543397815 expand 
881 0.0503922489278784 0.00387047186947708 0.0314865135279323 0.000158689798965779 0.018566334390642 0.00419893194124576 -2007.61543397815 reflect 
882 0.0503922489278784 0.00387047186947708 0.0314865135279323 0.000158689798965779 0.018566334390642 0.00419893194124576 -2007.61543397815 reflect 
883 0.0503922489278784 0.00387047186947708 0.0314865135279323 0.000158689798965779 0.018566334390642 0.00419893194124576 -2007.61543397815 reflect 
884 0.0503922489278784 0.00387047186947708 0.0314865135279323 0.000158689798965779 0.018566334390642 0.00419893194124576 -2007.61543397815 reflect 
885 0.0503922489278784 0.00387047186947708 0.0314865135279323 0.000158689798965779 0.018566334390642 0.00419893194124576 -2007.61543397815 contract inside 
886 0.0503922805278715 0.003870333583262 0.031486543080875 0.000158693768841178 0.0185663367097896 0.0041

936 0.0503927279111545 0.00386594776536157 0.0314868763743362 0.000159312721068797 0.018566939353479 0.00419917260121689 -2007.61543383249 expand 
937 0.0503927279111545 0.00386594776536157 0.0314868763743362 0.000159312721068797 0.018566939353479 0.00419917260121689 -2007.61543383249 reflect 
938 0.0503927279111545 0.00386594776536157 0.0314868763743362 0.000159312721068797 0.018566939353479 0.00419917260121689 -2007.61543383249 reflect 
939 0.0503927279111545 0.00386594776536157 0.0314868763743362 0.000159312721068797 0.018566939353479 0.00419917260121689 -2007.61543383249 reflect 
940 0.0503927279111545 0.00386594776536157 0.0314868763743362 0.000159312721068797 0.018566939353479 0.00419917260121689 -2007.61543383249 reflect 
941 0.0503927279111545 0.00386594776536157 0.0314868763743362 0.000159312721068797 0.018566939353479 0.00419917260121689 -2007.61543383249 contract inside 
942 0.0503927279111545 0.00386594776536157 0.0314868763743362 0.000159312721068797 0.018566939353479 0.00

992 0.050392043329878 0.00384642150268159 0.0314938275887199 0.000181677531076306 0.0185653030768245 0.00419800896270531 -2007.61542786738 contract outside 
993 0.050392043329878 0.00384642150268159 0.0314938275887199 0.000181677531076306 0.0185653030768245 0.00419800896270531 -2007.61542786738 reflect 
994 0.0503922193988739 0.00385219413275888 0.0314928913419683 0.000180040225301414 0.0185644103729285 0.00419844157375963 -2007.61542699068 expand 
995 0.0503922193988739 0.00385219413275888 0.0314928913419683 0.000180040225301414 0.0185644103729285 0.00419844157375963 -2007.61542699068 reflect 
996 0.0503922193988739 0.00385219413275888 0.0314928913419683 0.000180040225301414 0.0185644103729285 0.00419844157375963 -2007.61542699068 reflect 
997 0.0503922193988739 0.00385219413275888 0.0314928913419683 0.000180040225301414 0.0185644103729285 0.00419844157375963 -2007.61542699068 reflect 
998 0.0503922193988739 0.00385219413275888 0.0314928913419683 0.000180040225301414 0.018564410372928

1047 0.0503916011663921 0.00384541182729442 0.0314821584819819 0.000233610026312452 0.018563120439714 0.00419903414776297 -2007.6154113054 reflect 
1048 0.0503904904416816 0.00384487166372561 0.0314825719537737 0.000242351782340089 0.0185622464611955 0.00419900407362838 -2007.61541127852 contract inside 
1049 0.0503904904416816 0.00384487166372561 0.0314825719537737 0.000242351782340089 0.0185622464611955 0.00419900407362838 -2007.61541127852 contract inside 
1050 0.0503904904416816 0.00384487166372561 0.0314825719537737 0.000242351782340089 0.0185622464611955 0.00419900407362838 -2007.61541127852 reflect 
1051 0.050391285357263 0.00384147219217616 0.0314823981336349 0.000241684843460929 0.0185629036633342 0.00419867666084186 -2007.61541121299 contract inside 
1052 0.050391285357263 0.00384147219217616 0.0314823981336349 0.000241684843460929 0.0185629036633342 0.00419867666084186 -2007.61541121299 contract outside 
1053 0.050391285357263 0.00384147219217616 0.0314823981336349 0.0002416

1100 0.0503915641425558 0.00384242323813377 0.0314826207580683 0.000239826652182118 0.0185626735898827 0.00419890892642241 -2007.6154111158 reflect 
1101 0.0503915641425558 0.00384242323813377 0.0314826207580683 0.000239826652182118 0.0185626735898827 0.00419890892642241 -2007.6154111158 reflect 
1102 0.0503915641425558 0.00384242323813377 0.0314826207580683 0.000239826652182118 0.0185626735898827 0.00419890892642241 -2007.6154111158 reflect 
1103 0.0503915838284253 0.0038424376189539 0.0314826051021919 0.000239747514182674 0.0185626798261892 0.00419890609551306 -2007.61541111575 contract inside 
1104 0.0503915838284253 0.0038424376189539 0.0314826051021919 0.000239747514182674 0.0185626798261892 0.00419890609551306 -2007.61541111575 contract inside 
1105 0.0503915893113225 0.00384230980716391 0.0314826408338093 0.00023980221081511 0.0185626829487053 0.00419890615812744 -2007.61541111575 reflect 
1106 0.0503915893113225 0.00384230980716391 0.0314826408338093 0.00023980221081511 0.01856

### ETD model (Focal trait)

In [28]:
### ETD model (Focal trait)

In [29]:
idparslist <- id_paramPos(traits, num_concealed_states = 3)
idparslist[[1]][c(1,4,7)] <- 1 #Speciation varies only between the 3 reprod states
idparslist[[1]][c(2,5,8)] <- 2 
idparslist[[1]][c(3,6,9)] <- 3 
idparslist[[2]][] <- 4 #same extinction rate
diag(idparslist[[3]]) <- NA#remove same state transition rate (diagonal)
#all transition rates equal
idparslist[[3]][1,c(2,4,7)] <- 5
idparslist[[3]][1,c(3,5,6,8,9)] <- 0
idparslist[[3]][2,c(1,3,5,8)] <- 5
idparslist[[3]][2,c(4,6,7,9)] <- 0
idparslist[[3]][3,c(2,6,9)] <- 5
idparslist[[3]][3,c(1,4,5,7,8)] <- 0
idparslist[[3]][4,c(1,5,7)] <- 5
idparslist[[3]][4,c(2,3,6,8,9)] <- 0
idparslist[[3]][5,c(2,4,6,8)] <- 5
idparslist[[3]][5,c(1,3,7,9)] <- 0
idparslist[[3]][6,c(3,5,9)] <- 5
idparslist[[3]][6,c(1,2,4,7,8)] <- 0
idparslist[[3]][7,c(1,4,8)] <- 5
idparslist[[3]][7,c(2,3,5,6,9)] <- 0
idparslist[[3]][8,c(2,5,7,9)] <- 5
idparslist[[3]][8,c(1,3,4,6)] <- 0
idparslist[[3]][9,c(3,6,8)] <- 5
idparslist[[3]][9,c(1,2,4,5,7)] <- 0
idparslist[[3]]

,1A,2A,3A,1B,2B,3B,1C,2C,3C
1A,NA,5,0,5,0,0,5,0,0
2A,5,NA,5,0,5,0,0,5,0
3A,0,5,NA,0,0,5,0,0,5
1B,5,0,0,NA,5,0,5,0,0
2B,0,5,0,5,NA,5,0,5,0
3B,0,0,5,0,5,NA,0,0,5
1C,5,0,0,5,0,0,NA,5,0
2C,0,5,0,0,5,0,5,NA,5
3C,0,0,5,0,0,5,0,5,NA


In [30]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda, 3),intGuessMu,0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:5)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

ETD_sp<-secsse_ml(phy2.0,traits, num_concealed_states=3, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2216.778 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2216.77796730219 initial 
2 0.0290312549170369 0.0290312549170369 0.0290312549170369 0.0100156958161729 0.225728155339806 -2214.33256959003 expand 
3 0.0290312549170369 0.0290312549170369 0.0290312549170369 0.0100156958161729 0.225728155339806 -2214.33256959003 reflect 
4 0.0290312549170369 0.0290312549170369 0.0290312549170369 0.0100156958161729 0.225728155339806 -2214.33256959003 reflect 
5 0.0308410939666489 0.0308410939666489 0.0271760673764524 0.00965872124701668 0.221553525814692 -2212.29908030112 expand 
6 0.0308410939666489 0.0308410939666489 0.0271760673764524 0.00965872124701668 0.221553525814692 -2212.29908030112 reflect 
7 0.0304637794711892 0.0321061920803834 0.02986475

63 0.0409740901267415 0.0452319664261749 0.022095461701034 0.0106389136728819 0.00413857193723503 -2021.10841359312 contract inside 
64 0.0409740901267415 0.0452319664261749 0.022095461701034 0.0106389136728819 0.00413857193723503 -2021.10841359312 contract inside 
65 0.0406917897710576 0.0449665564873849 0.0226407185745447 0.0106318341906891 0.00404663369398894 -2021.0947828039 reflect 
66 0.0408806940761304 0.0455642963165632 0.0217074166386726 0.0106654242616058 0.00423637131013812 -2021.02415838132 reflect 
67 0.0403484798390882 0.0451346727616637 0.0226243024123605 0.0106583656793353 0.00435933196851018 -2020.92699075126 expand 
68 0.0403484798390882 0.0451346727616637 0.0226243024123605 0.0106583656793353 0.00435933196851018 -2020.92699075126 reflect 
69 0.0403484798390882 0.0451346727616637 0.0226243024123605 0.0106583656793353 0.00435933196851018 -2020.92699075126 reflect 
70 0.0401096428139014 0.045659086386029 0.0220914099022592 0.0107038413355545 0.00423274631614814 -2020.71

127 0.0378521798616606 0.0469517451189732 0.0218682784921048 0.0108868199754089 0.0042488248688628 -2020.26330206505 expand 
128 0.0378839684594824 0.0468597244565543 0.0219692298170185 0.0108791328220301 0.00424722164726442 -2020.26165337499 expand 
129 0.0378839684594824 0.0468597244565543 0.0219692298170185 0.0108791328220301 0.00424722164726442 -2020.26165337499 reflect 
130 0.0377568376925221 0.046986757663641 0.0218827390987703 0.0108927958741631 0.00423616511658403 -2020.2608116693 expand 
131 0.0379514423390227 0.0469441882560694 0.0217982096391713 0.0108804466608948 0.00423125823936067 -2020.25954301975 expand 
132 0.0378930499323829 0.0470127890412232 0.0217376642887681 0.0108868662371191 0.0042626352158514 -2020.25860267978 expand 
133 0.0379190888165371 0.0469434602674125 0.0218100541130364 0.0108805163863781 0.004234482931964 -2020.25283544059 expand 
134 0.0378683585948734 0.0469652025098749 0.021805650746264 0.0108831499585628 0.00424117500023893 -2020.24743476444 expand

193 0.0356808786184711 0.0371899621429758 0.0183914804696353 0.00825558442293329 0.00400159140155199 -2014.90419449519 reflect 
194 0.0356808786184711 0.0371899621429758 0.0183914804696353 0.00825558442293329 0.00400159140155199 -2014.90419449519 contract inside 
195 0.0331438862021384 0.0377869310579961 0.0179059225050363 0.00824884843354201 0.00438476341656952 -2014.73910173833 reflect 
196 0.0331438862021384 0.0377869310579961 0.0179059225050363 0.00824884843354201 0.00438476341656952 -2014.73910173833 reflect 
197 0.0335580904863592 0.0370984886425036 0.0169820716938167 0.00800507487907901 0.00446856727341695 -2014.60898290331 reflect 
198 0.0335580904863592 0.0370984886425036 0.0169820716938167 0.00800507487907901 0.00446856727341695 -2014.60898290331 reflect 
199 0.0335580904863592 0.0370984886425036 0.0169820716938167 0.00800507487907901 0.00446856727341695 -2014.60898290331 reflect 
200 0.0344556609014071 0.0384571969255698 0.0166881943065999 0.00830225679494784 0.0041685373837

255 0.0340797423857947 0.0372104771538858 0.0167077862132417 0.00801451534839782 0.00418647633457153 -2014.45560521328 contract outside 
256 0.034074989878397 0.0372112701371897 0.0167082247651038 0.00801457654722064 0.00418627171539298 -2014.45560426879 contract inside 
257 0.034074989878397 0.0372112701371897 0.0167082247651038 0.00801457654722064 0.00418627171539298 -2014.45560426879 reflect 
258 0.034074989878397 0.0372112701371897 0.0167082247651038 0.00801457654722064 0.00418627171539298 -2014.45560426879 contract inside 
259 0.034074989878397 0.0372112701371897 0.0167082247651038 0.00801457654722064 0.00418627171539298 -2014.45560426879 contract outside 
260 0.034074989878397 0.0372112701371897 0.0167082247651038 0.00801457654722064 0.00418627171539298 -2014.45560426879 contract inside 
261 0.034074989878397 0.0372112701371897 0.0167082247651038 0.00801457654722064 0.00418627171539298 -2014.45560426879 contract outside 
262 0.034074989878397 0.0372112701371897 0.0167082247651038

...

383 0.034085811040955 0.0372059394110823 0.0167116117528466 0.00801480621749349 0.00419162169178181 -2014.45553114323 reflect 
384 0.0340889720092658 0.0372161395041871 0.0167210224428093 0.00801870903099227 0.00419451186091062 -2014.45551728803 expand 
385 0.0340806199685047 0.037203737802475 0.0167045433724136 0.00801357689752444 0.00419284662285796 -2014.45549809653 expand 
386 0.0340806199685047 0.037203737802475 0.0167045433724136 0.00801357689752444 0.00419284662285796 -2014.45549809653 reflect 
387 0.0340748335859193 0.0372147449296835 0.016712978716665 0.00801680629723592 0.0041922122954559 -2014.45547735578 expand 
388 0.0340775347909508 0.0372040162218075 0.016712285646592 0.00801501433757356 0.00419553340548067 -2014.45544662994 expand 
389 0.0340775347909508 0.0372040162218075 0.016712285646592 0.00801501433757356 0.00419553340548067 -2014.45544662994 reflect 
390 0.0340775347909508 0.0372040162218075 0.016712285646592 0.00801501433757356 0.00419553340548067 -2014.455446629

448 0.034001840082585 0.036908146291736 0.016910821526353 0.00809399532362895 0.00421754529551107 -2014.44314990539 reflect 
449 0.0340400651539635 0.0369187698811992 0.0170054182272949 0.0081266484199978 0.00420042431063946 -2014.44060710562 expand 
450 0.0340400651539635 0.0369187698811992 0.0170054182272949 0.0081266484199978 0.00420042431063946 -2014.44060710562 reflect 
451 0.0340400651539635 0.0369187698811992 0.0170054182272949 0.0081266484199978 0.00420042431063946 -2014.44060710562 reflect 
452 0.0340400651539635 0.0369187698811992 0.0170054182272949 0.0081266484199978 0.00420042431063946 -2014.44060710562 reflect 
453 0.0340186034766435 0.0368179889333686 0.0169194460464303 0.00810131220967779 0.00418996079959445 -2014.44059906534 reflect 
454 0.0340186034766435 0.0368179889333686 0.0169194460464303 0.00810131220967779 0.00418996079959445 -2014.44059906534 contract inside 
455 0.0340186034766435 0.0368179889333686 0.0169194460464303 0.00810131220967779 0.00418996079959445 -20

513 0.0342041372132146 0.0368779468148151 0.01715846738265 0.00844917321547858 0.00424890981839051 -2014.39764403978 reflect 
514 0.0342041372132146 0.0368779468148151 0.01715846738265 0.00844917321547858 0.00424890981839051 -2014.39764403978 reflect 
515 0.0342041372132146 0.0368779468148151 0.01715846738265 0.00844917321547858 0.00424890981839051 -2014.39764403978 reflect 
516 0.0341414564974733 0.0369129350573091 0.0172927039867536 0.00864358321204275 0.00430551392030939 -2014.39653476706 reflect 
517 0.0344128970856646 0.0367812230723589 0.0172042474497155 0.00869522767724154 0.00431872476927116 -2014.38905001994 expand 
518 0.0343916398815928 0.0370880776471598 0.017522029403329 0.00873462808280322 0.00427227829526914 -2014.37595988826 expand 
519 0.0343916398815928 0.0370880776471598 0.017522029403329 0.00873462808280322 0.00427227829526914 -2014.37595988826 reflect 
520 0.0342492341928857 0.0370006616880285 0.0172801070408884 0.00885247748154975 0.00430602056560686 -2014.3626443

577 0.0367378892405453 0.0388187322693093 0.0193015187385149 0.0123701054966267 0.00415691601424147 -2014.09132099102 contract inside 
578 0.0367571755073658 0.0388460837941841 0.0192865619980819 0.0123608264284382 0.00415927325690655 -2014.09128614726 contract inside 
579 0.0367571755073658 0.0388460837941841 0.0192865619980819 0.0123608264284382 0.00415927325690655 -2014.09128614726 reflect 
580 0.0367571755073658 0.0388460837941841 0.0192865619980819 0.0123608264284382 0.00415927325690655 -2014.09128614726 contract inside 
581 0.0367571755073658 0.0388460837941841 0.0192865619980819 0.0123608264284382 0.00415927325690655 -2014.09128614726 contract inside 
582 0.0367571755073658 0.0388460837941841 0.0192865619980819 0.0123608264284382 0.00415927325690655 -2014.09128614726 reflect 
583 0.0367571755073658 0.0388460837941841 0.0192865619980819 0.0123608264284382 0.00415927325690655 -2014.09128614726 contract outside 
584 0.0367478136280565 0.0388171826102026 0.019278355691646 0.01234446

640 0.0367544706459785 0.0388251264120201 0.0192889056383941 0.0123599684471368 0.00415921951511792 -2014.09125715316 reflect 
641 0.0367544706459785 0.0388251264120201 0.0192889056383941 0.0123599684471368 0.00415921951511792 -2014.09125715316 reflect 
642 0.0367544706459785 0.0388251264120201 0.0192889056383941 0.0123599684471368 0.00415921951511792 -2014.09125715316 contract inside 
643 0.0367544706459785 0.0388251264120201 0.0192889056383941 0.0123599684471368 0.00415921951511792 -2014.09125715316 reflect 
644 0.036754516908321 0.0388251236784343 0.0192889684663843 0.0123600325669435 0.00415922002507143 -2014.09125715314 contract inside 
645 0.0367544813226012 0.0388252086224301 0.0192889976769404 0.0123600263063012 0.0041592243068683 -2014.09125715292 contract inside 
Optimization has terminated successfully. 


### Saving Data

In [31]:
### Saving Data

In [32]:
save(CR_sp, CTD2_sp, CTD3_sp, CTD4_sp, ETD_sp, file = "Body_size_data.rds")

### Comparing models

In [33]:
### Comparing models

In [34]:
# Aicc function
aicc<-function(ll,k){round((2*(-round(ll,4))+2*k+(2*k*(k+1))/(Ntip(phy2.0)-k-1)),3)}

In [35]:
aicc_vec<-c(aicc(CR_sp$ML, 3), aicc(CTD2_sp$ML, 4), aicc(CTD3_sp$ML, 5), aicc(CTD4_sp$ML, 6), aicc(ETD_sp$ML, 5))

In [36]:
body_size_model_comp<-as.data.frame(cbind(c(CR_sp$ML, CTD2_sp$ML, CTD3_sp$ML, CTD4_sp$ML, ETD_sp$ML), c(3, 4, 5, 6, 5), aicc_vec, round(akaike.weights(aicc_vec)$deltaAIC, 3), round(akaike.weights(aicc_vec)$weights, 3)))

In [49]:
colnames(body_size_model_comp)<-c("log likelihood", "number of parameters", "AICc", "Delta AIC", "AIC weights")
rownames(body_size_model_comp)<-c("Constant rate model", "Conceal trait 2", "Conceal trait 3", "Conceal trait 4", "Focal trait")

### Saving Data

In [50]:
### Saving Data

In [52]:
write.table(body_size_model_comp, "Path/to/your/exit.file", sep ="\t")